In [1]:
import torchvision.models as models
import torchvision.transforms as trans
import torchvision.datasets as dset
import torch
from robustness import model_utils
import dill
from robustness import model_utils, datasets
from user_constants import DATA_PATH_DICT

In [2]:
CIFAR_SIZE = 32
CIFAR_MEAN = [0.4914, 0.4822, 0.4465]
CIFAR_STD = [0.2023, 0.1994, 0.2010]

# applies the normalization transformations
def apply_normalization(imgs, dataset):
    if dataset == 'imagenet':
        mean = IMAGENET_MEAN
        std = IMAGENET_STD
    elif dataset == 'cifar':
        mean = CIFAR_MEAN
        std = CIFAR_STD
    elif dataset == 'mnist':
        mean = MNIST_MEAN
        std = MNIST_STD
    else:
        mean = [0, 0, 0]
        std = [1, 1, 1]
    imgs_tensor = imgs.clone()
    if dataset == 'mnist':
        imgs_tensor = (imgs_tensor - mean[0]) / std[0]
    else:
        if imgs.dim() == 3:
            for i in range(imgs_tensor.size(0)):
                imgs_tensor[i, :, :] = (imgs_tensor[i, :, :] - mean[i]) / std[i]
        else:
            for i in range(imgs_tensor.size(1)):
                imgs_tensor[:, i, :, :] = (imgs_tensor[:, i, :, :] - mean[i]) / std[i]
    return imgs_tensor

In [3]:
DATA = 'CIFAR'
dataset_function = getattr(datasets, DATA)
dataset = dataset_function(DATA_PATH_DICT[DATA])

# Load model
model_kwargs = {
    'arch': 'resnet50',
    'dataset': dataset,
    'resume_path': f'./models/cifar_std.pt'
}

model, _ = model_utils.make_and_restore_model(**model_kwargs)
model.cuda()
model.eval()
print('loading done')

=> loading checkpoint './models/cifar_std.pt'
=> loaded checkpoint './models/cifar_std.pt' (epoch 190)
loading done


In [25]:

CIFAR_TRANSFORM = trans.Compose([trans.ToTensor()])
testset = dset.ImageFolder('./cifar10/smalltest', CIFAR_TRANSFORM)

image_size = 32
images = torch.zeros(10, 3, image_size, image_size)
labels = torch.zeros(10).long()
predictions = torch.zeros(10).long()

idx = torch.arange(0, images.size(0)).long()
for i in list(idx):
    images[i], labels[i] = testset[i]

# input_images = apply_normalization(images, 'cifar')[:5]

# with torch.no_grad():
#     output = torch.nn.Softmax().forward(model.forward(input_images.cuda()))
# prob, pred = output.max(1)

    
def get_probs(model, x, y):
    with torch.no_grad():
        output, _ = model(x.cuda())
    probs = torch.index_select(torch.nn.Softmax()(output).cpu(), 1, y)
    return torch.diag(probs)

def get_preds(model, x):
    # Don't need to normalize again since we use robustness's model
    # output = model(normalize(torch.autograd.Variable(x.cuda()))).cpu()
    with torch.no_grad():
        output, _ = model(x.cuda())
    _, preds = output.cpu().max(1)
    # _, preds = output.data.max(1)
    return preds


with torch.no_grad():
    output, _ = model(images.cuda())

probs1 = torch.nn.Softmax(dim=0)(output).cpu()
probs2 = torch.nn.Softmax(dim=1)(output).cpu()
print(probs1.sum(dim=0))
print(probs2.sum(dim=0))

tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000,
        1.0000])
tensor([1.0011, 0.9969, 0.9790, 1.0212, 0.9986, 1.0018, 1.0003, 1.0004, 1.0001,
        1.0007])
